# Chaii data training

Trainig using MuRIL

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec  2 05:45:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets

     |████████████████████████████████| 298 kB 15.0 MB/s 
     |████████████████████████████████| 243 kB 61.4 MB/s 
     |████████████████████████████████| 61 kB 667 kB/s 
     |████████████████████████████████| 1.1 MB 60.3 MB/s 
     |████████████████████████████████| 132 kB 64.5 MB/s 
     |████████████████████████████████| 271 kB 67.3 MB/s 
     |████████████████████████████████| 160 kB 57.0 MB/s 
     |████████████████████████████████| 192 kB 72.0 MB/s 


In [3]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 15.7 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 3.3 MB 52.3 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
# importing the dependencies
EXP_NAME = 'pretraining_muril'
FOLDER_NAME = EXP_NAME
SEED = 68

In [5]:
import torch
torch.cuda.is_available()

True

In [6]:
#!pip install kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 5.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=514cb7fef116d7d2bdcf76a9c9c5fb4ac3155ba60596b350273a08af132a6097
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [7]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle kernels output anubhavsrivastava/5foldxlm-r -p /content/5foldxlmr

Output file downloaded to /content/5foldxlmr/output/checkpoint-fold-0/config.json
tcmalloc: large alloc 2239750144 bytes == 0x55dd57406000 @  0x7fc6573781e7 0x55dcb2e5bf98 0x55dcb2e26e27 0x55dcb2e28e20 0x55dcb2e2a2ed 0x55dcb2f1be1d 0x55dcb2e9de99 0x55dcb2d6ad14 0x55dcb2f1bf31 0x55dcb2f49849 0x55dcb2e99a7d 0x55dcb2e989ee 0x55dcb2e2c48c 0x55dcb2e2c698 0x55dcb2e9afe4 0x55dcb2e989ee 0x55dcb2e2c48c 0x55dcb2e2c698 0x55dcb2e9afe4 0x55dcb2e989ee 0x55dcb2d6ae2b 0x55dcb2e9afe4 0x55dcb2e989ee 0x55dcb2e2bbda 0x55dcb2e9dd00 0x55dcb2e989ee 0x55dcb2e2bbda 0x55dcb2e99c0d 0x55dcb2e989ee 0x55dcb2e2c48c 0x55dcb2e2c698
Output file downloaded to /content/5foldxlmr/output/checkpoint-fold-0/pytorch_model.bin
Output file downloaded to /content/5foldxlmr/output/checkpoint-fold-0/result_dict.json
Output file downloaded to /content/5foldxlmr/output/checkpoint-fold-0/sentencepiece.bpe.model
Output file downloaded to /content/5foldxlmr/output/checkpoint-fold-0/special_tokens_map.json
Output file downloaded to /con

In [9]:
import os

import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler

from datasets import Dataset
from sklearn import model_selection 
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

from tqdm.autonotebook import tqdm

import gc

import re
import random
random.seed(SEED)

import warnings
warnings.filterwarnings("ignore")

In [10]:
model_checkpoint = "google/muril-base-cased"
batch_size = 8
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
max_length = 384
doc_stride = 128
pad_on_right = tokenizer.padding_side == "right"
n_folds = 3

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

## Processing the features

In [11]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


In [12]:
# Create data folds
def create_folds(data, num_splits):
    data["kfold"] = -1
    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=2021)
    for fold_num, (t_, v_) in enumerate(kf.split(X=data, y=data.language.values)):
        data.loc[v_, "kfold"] = fold_num
    return data

def convert_answers(row):
    return {"answer_start": [row[0]], "text": [row[1]]}


In [13]:
def negative_sampling(examples, ratio=0.1):
    
    def _sample(pos):
        if pos != 0: return True
        else: return random.random() < ratio
                
    indices = [i for i,x in enumerate(examples['start_positions']) if _sample(x)]

    for key in examples.keys():
        examples[key] = [x for i,x in enumerate(examples[key]) if i in indices]
        
    return examples

## Create training and validation

In [14]:
!mkdir datasets

In [15]:
#bring data from drive
!cp drive/MyDrive/IR/mlqa_hindi.csv /content
!cp drive/MyDrive/IR/squad_translated_tamil.csv /content
!cp drive/MyDrive/IR/xquad.csv /content
!cp drive/MyDrive/IR/test.csv  /content
!cp drive/MyDrive/IR/train.csv /content

In [ ]:
# Import the files
extended_hi = '/content/xquad.csv'
hi_data = pd.read_csv(extended_hi)

# Tamil
extended_ta = '/content/squad_translated_tamil.csv'
ta_data = pd.read_csv(extended_ta)

# Edit the tamil dataset:
ta_data['language'] = 'tamil'

In [ ]:
# concatenate the data:
ext_data = pd.concat([hi_data, ta_data])
ext_data.head()

,context,question,answer_text,answer_start,language
0,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,पैंथर्स डिफ़ेंस ने कितने अंक दिए?,308,35.0,hindi
1,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जेरेड एलन के पास कितने करियर सैक थे?,136,380.0,hindi
2,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,ल्यूक कुएक्ली ने कितने टैकल रजिस्टर किए?,118,743.0,hindi
3,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जोश नॉर्मन ने कितने बॉल को इंटरसेप्ट किया?,चार,90.0,hindi
4,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,इस सीज़न में टीम से किसने सबसे अधिक सैक रजिस्टर...,कावन शॉर्ट,169.0,hindi


In [ ]:
# ad an id column:
index = 0
col = list(ext_data.index)
ext_data.insert(loc=index, column='id', value=col)
ext_data.head()

,id,context,question,answer_text,answer_start,language
0,0,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,पैंथर्स डिफ़ेंस ने कितने अंक दिए?,308,35.0,hindi
1,1,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जेरेड एलन के पास कितने करियर सैक थे?,136,380.0,hindi
2,2,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,ल्यूक कुएक्ली ने कितने टैकल रजिस्टर किए?,118,743.0,hindi
3,3,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जोश नॉर्मन ने कितने बॉल को इंटरसेप्ट किया?,चार,90.0,hindi
4,4,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,इस सीज़न में टीम से किसने सबसे अधिक सैक रजिस्टर...,कावन शॉर्ट,169.0,hindi


In [ ]:
# Split into training and validation
# Shuffle the data
idx = np.arange(len(ext_data))
np.random.seed(111)
np.random.shuffle(idx)

ext_data = ext_data.iloc[idx[:]]

# Reset the indices:
ext_data = ext_data.reset_index(drop=True)


In [16]:
# Process the dataset
# Convert the answers:
def convert_answers(row):
    return {'answer_start': [row[0]], 'text': [row[1]]}

# Helper functions
def create_folds(data, num_splits):
    data["kfold"] = -1
    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=2021)
    for fold_num, (t_, v_) in enumerate(kf.split(X=data, y=data.language.values)):
        data.loc[v_, "kfold"] = fold_num
    return data

ext_data['answers'] = ext_data[['answer_start', 'answer_text']].apply(convert_answers, axis=1)
ext_data = create_folds(ext_data, 3)

fold = 0

train_set = ext_data[ext_data.kfold != fold]
val_set = ext_data[ext_data.kfold == fold]

NameError: ignored

In [ ]:
train_data = Dataset.from_pandas(train_set)
valid_data = Dataset.from_pandas(val_set)

## Tokenize the features

In [ ]:
# Tokenizing 
train_features = train_data.map(prepare_train_features, batched=True, remove_columns=train_data.column_names)
valid_features = valid_data.map(prepare_train_features, batched=True, remove_columns=valid_data.column_names)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
# Negative sampling
train_str1 = train_features.map(negative_sampling, fn_kwargs={'ratio':0.1}, batched=True, batch_size=8)
train_str2 = train_features.map(negative_sampling, fn_kwargs={'ratio':0.2}, batched=True, batch_size=8)

  0%|          | 0/404 [00:00<?, ?ba/s]

  0%|          | 0/404 [00:00<?, ?ba/s]

In [ ]:
from datasets import concatenate_datasets
tokenized_train_all = concatenate_datasets([train_str1, train_str2])

## Actual training of the model

In [ ]:
from datasets import load_metric
metric = load_metric("f1")

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import DataLoader, Dataset, IterableDataset, RandomSampler, SequentialSampler

from transformers.file_utils import is_datasets_available

class MyTrainer(Trainer): 

    def get_train_dataloader(self) -> DataLoader:
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")

        train_dataset = self.train_dataset
        if is_datasets_available() and isinstance(train_dataset, Dataset):
            train_dataset = self._remove_unused_columns(train_dataset, description="training")

        train_sampler = SequentialSampler(self.train_dataset)

        return DataLoader(
            train_dataset,
            batch_size=self.args.train_batch_size,
            sampler=train_sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
        )

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/909M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the m

In [ ]:
args = TrainingArguments(
    f"chaii-qa-{EXP_NAME}",
    evaluation_strategy = "steps",
    logging_strategy = "steps",
    logging_steps = 100,
    save_steps = 100,
    save_strategy = "steps",
    learning_rate=3e-4,
    gradient_accumulation_steps=4,
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to='none',
    save_total_limit=15
)

data_collator = default_data_collator

trainer = MyTrainer(
    model,
    args,
    train_dataset=tokenized_train_all,
    eval_dataset=valid_features,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()
trainer.save_model(f"{FOLDER_NAME}/final")

***** Running training *****
  Num examples = 6370
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 597


Step,Training Loss,Validation Loss
100,5.478500,4.841656
200,4.116600,3.564932
300,2.701100,3.181524
400,1.545800,3.123061
500,0.774300,3.236356


***** Running Evaluation *****
  Num examples = 1613
  Batch size = 8
Saving model checkpoint to chaii-qa-pretraining_muril/checkpoint-100
Configuration saved in chaii-qa-pretraining_muril/checkpoint-100/config.json
Model weights saved in chaii-qa-pretraining_muril/checkpoint-100/pytorch_model.bin
tokenizer config file saved in chaii-qa-pretraining_muril/checkpoint-100/tokenizer_config.json
Special tokens file saved in chaii-qa-pretraining_muril/checkpoint-100/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1613
  Batch size = 8
Saving model checkpoint to chaii-qa-pretraining_muril/checkpoint-200
Configuration saved in chaii-qa-pretraining_muril/checkpoint-200/config.json
Model weights saved in chaii-qa-pretraining_muril/checkpoint-200/pytorch_model.bin
tokenizer config file saved in chaii-qa-pretraining_muril/checkpoint-200/tokenizer_config.json
Special tokens file saved in chaii-qa-pretraining_muril/checkpoint-200/special_tokens_map.json
***** Running Evaluati

In [ ]:
!zip -r chaii-qa-pretraining_muril.zip chaii-qa-pretraining_muril/

  adding: chaii-qa-pretraining_muril/ (stored 0%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/ (stored 0%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/pytorch_model.bin (deflated 7%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/config.json (deflated 49%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/special_tokens_map.json (deflated 40%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/rng_state.pth (deflated 27%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/tokenizer_config.json (deflated 37%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/vocab.txt (deflated 61%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/scheduler.pt (deflated 49%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/optimizer.pt (deflated 59%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/trainer_state.json (deflated 73%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/training_args.bin (deflated 48%)
  adding: chaii-qa-pretraining_muril/checkpoint-500/t

In [ ]:
!zip -r pretraining_muril.zip pretraining_muril/

  adding: pretraining_muril/ (stored 0%)
  adding: pretraining_muril/final/ (stored 0%)
  adding: pretraining_muril/final/pytorch_model.bin (deflated 7%)
  adding: pretraining_muril/final/config.json (deflated 49%)
  adding: pretraining_muril/final/special_tokens_map.json (deflated 40%)
  adding: pretraining_muril/final/tokenizer_config.json (deflated 37%)
  adding: pretraining_muril/final/vocab.txt (deflated 61%)
  adding: pretraining_muril/final/training_args.bin (deflated 48%)
  adding: pretraining_muril/final/tokenizer.json (deflated 62%)


In [ ]:
!zip -r pretraining_muril_2.zip pretraining_muril_2/


  adding: pretraining_muril_2/ (stored 0%)
  adding: pretraining_muril_2/final/ (stored 0%)
  adding: pretraining_muril_2/final/pytorch_model.bin (deflated 7%)
  adding: pretraining_muril_2/final/config.json (deflated 48%)
  adding: pretraining_muril_2/final/special_tokens_map.json (deflated 40%)
  adding: pretraining_muril_2/final/tokenizer_config.json (deflated 38%)
  adding: pretraining_muril_2/final/vocab.txt (deflated 61%)
  adding: pretraining_muril_2/final/training_args.bin (deflated 48%)
  adding: pretraining_muril_2/final/tokenizer.json (deflated 62%)


In [ ]:
!zip -r pretraining_muril_3.zip pretraining_muril_3/

  adding: pretraining_muril_3/ (stored 0%)
  adding: pretraining_muril_3/final/ (stored 0%)
  adding: pretraining_muril_3/final/pytorch_model.bin (deflated 7%)
  adding: pretraining_muril_3/final/config.json (deflated 48%)
  adding: pretraining_muril_3/final/special_tokens_map.json (deflated 40%)
  adding: pretraining_muril_3/final/tokenizer_config.json (deflated 38%)
  adding: pretraining_muril_3/final/vocab.txt (deflated 61%)
  adding: pretraining_muril_3/final/training_args.bin (deflated 48%)
  adding: pretraining_muril_3/final/tokenizer.json (deflated 62%)


In [ ]:
!cp chaii-qa-pretraining_muril.zip drive/MyDrive/IR

In [ ]:
!cp pretraining_muril.zip drive/MyDrive/IR

In [ ]:
!cp pretraining_muril_2.zip drive/MyDrive/IR

In [ ]:
!cp pretraining_muril_3.zip drive/MyDrive/IR

## Preparing validation data

In [ ]:
import collections

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples



In [ ]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [ ]:
def jaccard(row): 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
from string import punctuation

def postuning(s):
    s = " ".join(s.split())
    s = s.strip(punctuation)
    return s

In [ ]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(row):
    truth = row[0]
    prediction = row[1]
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(row):
    truth = row[0]
    prediction = row[1]
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [ ]:
train_data

Dataset({
    features: ['id', 'context', 'question', 'answer_text', 'answer_start', 'language', 'answers', 'kfold', '__index_level_0__'],
    num_rows: 3171
})

In [ ]:
validation_features = valid_data.map(prepare_validation_features,batched=True, remove_columns=valid_data.column_names)

valid_feats_small = validation_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

max_answer_length = 30

examples = valid_data
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

references = [{"id": ex["id"], "context": ex["context"], "question": ex["question"], "answer": ex["answer_text"]} for ex in valid_data]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1613 [00:00<?, ?ex/s]

In [ ]:
# final
raw_predictions = trainer.predict(valid_feats_small)
final_predictions = postprocess_qa_predictions(valid_data, validation_features, raw_predictions.predictions)
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res['postuned'] = res['prediction'].apply(postuning)
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
res.jaccard.mean(), res.pjaccard.mean()

***** Running Prediction *****
  Num examples = 1613
  Batch size = 8


Post-processing 1586 example predictions split into 1613 features.


  0%|          | 0/1586 [00:00<?, ?it/s]

(0.3692824969432347, 0.3584961521404965)

In [ ]:
res['em'] = res[['answer', 'prediction']].apply(compute_exact_match, axis=1)
res.em.mean()

0.30958385876418665

In [ ]:
res['f1'] = res[['answer', 'prediction']].apply(compute_f1, axis=1)
res.f1.mean()

0.39818869670229873

## 5. Repeat the same for next 2 folds

In [ ]:
# Fold 1
from datasets import Dataset
fold = 1

train_set = ext_data[ext_data.kfold != fold]
val_set = ext_data[ext_data.kfold == fold]
train_data = Dataset.from_pandas(train_set)
valid_data = Dataset.from_pandas(val_set)

In [ ]:
# Change the model checkpoint
model_checkpoint = '/content/pretraining_muril/final'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
EXP_NAME = 'pretraining_muril_2'
FOLDER_NAME = EXP_NAME


Didn't find file /content/pretraining_muril/final/added_tokens.json. We won't load it.
loading file /content/pretraining_muril/final/vocab.txt
loading file /content/pretraining_muril/final/tokenizer.json
loading file None
loading file /content/pretraining_muril/final/special_tokens_map.json
loading file /content/pretraining_muril/final/tokenizer_config.json


In [ ]:
# Repeat the same
# Tokenizing 
train_features = train_data.map(prepare_train_features, batched=True, remove_columns=train_data.column_names)
valid_features = valid_data.map(prepare_train_features, batched=True, remove_columns=valid_data.column_names)

# Negative sampling
train_str1 = train_features.map(negative_sampling, fn_kwargs={'ratio':0.1}, batched=True, batch_size=8)
train_str2 = train_features.map(negative_sampling, fn_kwargs={'ratio':0.2}, batched=True, batch_size=8)

from datasets import concatenate_datasets
tokenized_train_all = concatenate_datasets([train_str1, train_str2])

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

args = TrainingArguments(
    f"chaii-qa-{EXP_NAME}",
    evaluation_strategy = "steps",
    logging_strategy = "steps",
    logging_steps = 100,
    save_steps = 600,
    save_strategy = "steps",
    learning_rate=3e-4,
    gradient_accumulation_steps=4,
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to='none',
    save_total_limit=15
)

data_collator = default_data_collator

trainer = MyTrainer(
    model,
    args,
    train_dataset=tokenized_train_all,
    eval_dataset=valid_features,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/403 [00:00<?, ?ba/s]

  0%|          | 0/403 [00:00<?, ?ba/s]

loading configuration file /content/pretraining_muril/final/config.json
Model config BertConfig {
  "_name_or_path": "google/muril-base-cased",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 197285
}

loading weights file /content/pretraining_muril/final/pytorch_model.bin
All model checkpoint weights were used when initializing BertForQuestionAnswering.

All the weights of BertForQuestionAnswering were initialized from the model c

In [ ]:
trainer.train()
trainer.save_model(f"{FOLDER_NAME}/final")

***** Running training *****
  Num examples = 6372
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 597


Step,Training Loss,Validation Loss
100,1.743600,0.744197
200,1.011900,0.823065
300,0.555200,0.813104
400,0.294200,0.835072
500,0.154400,0.892290


***** Running Evaluation *****
  Num examples = 1618
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1618
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1618
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1618
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1618
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to pretraining_muril_2/final
Configuration saved in pretraining_muril_2/final/config.json
Model weights saved in pretraining_muril_2/final/pytorch_model.bin
tokenizer config file saved in pretraining_muril_2/final/tokenizer_config.json
Special tokens file saved in pretraining_muril_2/final/special_tokens_map.json


In [ ]:
validation_features = valid_data.map(prepare_validation_features,batched=True, remove_columns=valid_data.column_names)

valid_feats_small = validation_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

max_answer_length = 30

examples = valid_data
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

references = [{"id": ex["id"], "context": ex["context"], "question": ex["question"], "answer": ex["answer_text"]} for ex in valid_data]

# Round 2 results
# final
raw_predictions = trainer.predict(valid_feats_small)
final_predictions = postprocess_qa_predictions(valid_data, validation_features, raw_predictions.predictions)
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res['postuned'] = res['prediction'].apply(postuning)
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
res['em'] = res[['answer', 'prediction']].apply(compute_exact_match, axis=1)
res['f1'] = res[['answer', 'prediction']].apply(compute_f1, axis=1)

print("Model results:\n")
print(f"Jaccard Score: {res.jaccard.mean()}\nJaccard Score after post-tuning:{res.pjaccard.mean()}\nEM Score: {res.em.mean()}\nF1 Score: {res.f1.mean()}")



  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1618 [00:00<?, ?ex/s]

***** Running Prediction *****
  Num examples = 1618
  Batch size = 8


Post-processing 1586 example predictions split into 1618 features.


  0%|          | 0/1586 [00:00<?, ?it/s]

Model results:

Jaccard Score: 0.7161787764421004
Jaccard Score after post-tuning:0.6966037248998356
EM Score: 0.6765447667087011
F1 Score: 0.7347253375346626


## Fold 3

In [ ]:
from datasets import Dataset
fold = 2

train_set = ext_data[ext_data.kfold != fold]
val_set = ext_data[ext_data.kfold == fold]
train_data = Dataset.from_pandas(train_set)
valid_data = Dataset.from_pandas(val_set)

In [ ]:
# Change the model checkpoint
#model_checkpoint = '/content/pretraining_muril_2/final'
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model_checkpoint = '/content/pretraining_muril_3/final'
EXP_NAME = 'pretraining_muril_3'
FOLDER_NAME = EXP_NAME


In [ ]:
# Repeat the same
# Tokenizing 
train_features = train_data.map(prepare_train_features, batched=True, remove_columns=train_data.column_names)
valid_features = valid_data.map(prepare_train_features, batched=True, remove_columns=valid_data.column_names)

# Negative sampling
train_str1 = train_features.map(negative_sampling, fn_kwargs={'ratio':0.1}, batched=True, batch_size=8)
train_str2 = train_features.map(negative_sampling, fn_kwargs={'ratio':0.2}, batched=True, batch_size=8)

from datasets import concatenate_datasets
tokenized_train_all = concatenate_datasets([train_str1, train_str2])

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

args = TrainingArguments(
    f"chaii-qa-{EXP_NAME}",
    evaluation_strategy = "steps",
    logging_strategy = "steps",
    logging_steps = 100,
    save_steps = 600,
    save_strategy = "steps",
    learning_rate=3e-4,
    gradient_accumulation_steps=4,
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to='none',
    save_total_limit=15
)

data_collator = default_data_collator

trainer = MyTrainer(
    model,
    args,
    train_dataset=tokenized_train_all,
    eval_dataset=valid_features,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/404 [00:00<?, ?ba/s]

  0%|          | 0/404 [00:00<?, ?ba/s]

loading configuration file /content/pretraining_muril_3/final/config.json
Model config BertConfig {
  "_name_or_path": "/content/pretraining_muril_2/final",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 197285
}

loading weights file /content/pretraining_muril_3/final/pytorch_model.bin
All model checkpoint weights were used when initializing BertForQuestionAnswering.

All the weights of BertForQuestionAnswering were initialized f

In [ ]:
trainer.train()
trainer.save_model(f"{FOLDER_NAME}/final")


***** Running training *****
  Num examples = 6380
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 597


Step,Training Loss,Validation Loss
100,0.543400,0.240488
200,0.432100,0.254371
300,0.275200,0.305706
400,0.176100,0.318906
500,0.097600,0.250077


***** Running Evaluation *****
  Num examples = 1611
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1611
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1611
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1611
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1611
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to pretraining_muril_3/final
Configuration saved in pretraining_muril_3/final/config.json
Model weights saved in pretraining_muril_3/final/pytorch_model.bin
tokenizer config file saved in pretraining_muril_3/final/tokenizer_config.json
Special tokens file saved in pretraining_muril_3/final/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1611
  Batch size = 8


{'eval_loss': 0.2735559642314911, 'eval_runtime': 22.4268, 'eval_samples_per_second': 71.834, 'eval_steps_per_second': 9.007, 'epoch': 3.0}


In [ ]:
# em and f1 scores
fineval = trainer.evaluate()
print(fineval)

***** Running Evaluation *****
  Num examples = 3918
  Batch size = 8


{'eval_loss': 3.296325445175171, 'eval_runtime': 54.6281, 'eval_samples_per_second': 71.721, 'eval_steps_per_second': 8.97, 'epoch': 3.0}


In [ ]:
validation_features = valid_data.map(prepare_validation_features,batched=True, remove_columns=valid_data.column_names)

valid_feats_small = validation_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

max_answer_length = 30

examples = valid_data
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

references = [{"id": ex["id"], "context": ex["context"], "question": ex["question"], "answer": ex["answer_text"]} for ex in valid_data]

# Round 2 results
# final
raw_predictions = trainer.predict(valid_feats_small)
final_predictions = postprocess_qa_predictions(valid_data, validation_features, raw_predictions.predictions)
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res['postuned'] = res['prediction'].apply(postuning)
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
#res.jaccard.mean(), res.pjaccard.mean()
res['em'] = res[['answer', 'prediction']].apply(compute_exact_match, axis=1)
res['f1'] = res[['answer', 'prediction']].apply(compute_f1, axis=1)

print("Model results:\n")
print(f"Jaccard Score: {res.jaccard.mean()}\nJaccard Score after post-tuning:{res.pjaccard.mean()}\nEM Score: {res.em.mean()}\nF1 Score: {res.f1.mean()}")


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1611 [00:00<?, ?ex/s]

***** Running Prediction *****
  Num examples = 1611
  Batch size = 8


Post-processing 1585 example predictions split into 1611 features.


  0%|          | 0/1585 [00:00<?, ?it/s]

Model results:

Jaccard Score: 0.8135635892529567
Jaccard Score after post-tuning:0.7917076481383405
EM Score: 0.7962145110410095
F1 Score: 0.8212143503124889


In [ ]:
#!zip -r muril_weights.zip /content/pretraining_xlm_3

In [ ]:
#To bring back the weights from drive
!cp 'drive/MyDrive/IR/pretraining_muril_3.zip' pretraining_muril_3.zip
!unzip pretraining_muril_3.zip

Archive:  pretraining_muril_3.zip
   creating: pretraining_muril_3/
   creating: pretraining_muril_3/final/
  inflating: pretraining_muril_3/final/pytorch_model.bin  
  inflating: pretraining_muril_3/final/config.json  
  inflating: pretraining_muril_3/final/special_tokens_map.json  
  inflating: pretraining_muril_3/final/tokenizer_config.json  
  inflating: pretraining_muril_3/final/vocab.txt  
  inflating: pretraining_muril_3/final/training_args.bin  
  inflating: pretraining_muril_3/final/tokenizer.json  


In [ ]:
!ls -l

total 10675964
drwxr-xr-x 7 root root       4096 Dec  1 17:51 chaii-qa-pretraining_muril
drwxr-xr-x 2 root root       4096 Dec  1 18:19 chaii-qa-pretraining_muril_2
drwxr-xr-x 2 root root       4096 Dec  1 18:39 chaii-qa-pretraining_muril_3
-rw-r--r-- 1 root root 8285017350 Dec  1 18:07 chaii-qa-pretraining_muril.zip
drwxr-xr-x 2 root root       4096 Dec  1 17:32 datasets
drwx------ 6 root root       4096 Dec  1 17:58 drive
drwxr-xr-x 3 root root       4096 Dec  1 17:54 pretraining_muril
drwxr-xr-x 3 root root       4096 Dec  1 18:35 pretraining_muril_2
-rw-r--r-- 1 root root  882363748 Dec  1 19:10 pretraining_muril_2.zip
drwxr-xr-x 3 root root       4096 Dec  1 18:54 pretraining_muril_3
-rw-r--r-- 1 root root  882357321 Dec  1 19:11 pretraining_muril_3.zip
-rw-r--r-- 1 root root  882388608 Dec  1 18:13 pretraining_muril.zip
drwxr-xr-x 1 root root       4096 Nov 18 14:36 sample_data


Second step of training

In [ ]:
#bring data from drive
!cp drive/MyDrive/IR/mlqa_hindi.csv /content
!cp drive/MyDrive/IR/squad_translated_tamil.csv /content
!cp drive/MyDrive/IR/xquad.csv /content
!cp drive/MyDrive/IR/test.csv  /content
!cp drive/MyDrive/IR/train.csv /content

In [ ]:
#To bring back the weights from drive
!cp 'drive/MyDrive/IR/5foldxlmr.zip' 5foldxlmr.zip
!unzip 5foldxlmr.zip

Archive:  5foldxlmr.zip
   creating: 5foldxlmr/
  inflating: 5foldxlmr/5foldxlm-r.log  
   creating: 5foldxlmr/output/
   creating: 5foldxlmr/output/checkpoint-fold-1/
  inflating: 5foldxlmr/output/checkpoint-fold-1/config.json  
  inflating: 5foldxlmr/output/checkpoint-fold-1/pytorch_model.bin  
  inflating: 5foldxlmr/output/checkpoint-fold-1/special_tokens_map.json  
  inflating: 5foldxlmr/output/checkpoint-fold-1/tokenizer_config.json  
  inflating: 5foldxlmr/output/checkpoint-fold-1/result_dict.json  
  inflating: 5foldxlmr/output/checkpoint-fold-1/sentencepiece.bpe.model  
  inflating: 5foldxlmr/output/checkpoint-fold-1/tokenizer.json  
   creating: 5foldxlmr/output/checkpoint-fold-0/
  inflating: 5foldxlmr/output/checkpoint-fold-0/config.json  
  inflating: 5foldxlmr/output/checkpoint-fold-0/pytorch_model.bin  
  inflating: 5foldxlmr/output/checkpoint-fold-0/special_tokens_map.json  
  inflating: 5foldxlmr/output/checkpoint-fold-0/tokenizer_config.json  
  inflating: 5foldxlmr/ou

In [ ]:
def convert_answers(row):
    return {"answer_start": [row[0]], "text": [row[1]]}

In [17]:
import pandas as pd
#Read Data
# Import the files
chaii_train = '/content/train.csv'
train_df = pd.read_csv(chaii_train)

# Tamil
chaii_test = '/content/test.csv'
test_df = pd.read_csv(chaii_test)

train_df.head()

ext_data = train_df

# Preprocess the data
ext_data['answers'] = ext_data[['answer_start', 'answer_text']].apply(convert_answers, axis=1)


In [18]:
# Import the chaii dataset
chaii_data = pd.read_csv('/content/train.csv')

# Preprocess the data
chaii_data['answers'] = chaii_data[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

# Shuffle the data
chaii_data.head()

,id,context,question,answer_text,answer_start,language,answers
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"{'answer_start': [53], 'text': ['206']}"
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,"{'answer_start': [2358], 'text': ['காசுமீரில்']}"
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil,"{'answer_start': [0], 'text': ['சர் அலெக்ஸாண்ட..."
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil,"{'answer_start': [68], 'text': ['தாலாட்டு']}"
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil,"{'answer_start': [585], 'text': ['சூரியனும்']}"


In [19]:
# Split the data into training test:
# Setting the ratios
from sklearn.model_selection import train_test_split
full_train, test = train_test_split(chaii_data, test_size = 0.3)
full_train = full_train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [20]:
# Model
fold = 0
model_checkpoint = '/content/5foldxlmr/output/checkpoint-fold-2/'
FOLDER = f"chaii_fold_{fold}"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [21]:
def create_folds(data, num_splits):
    data["kfold"] = -1
    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=2021)
    for fold_num, (t_, v_) in enumerate(kf.split(X=data, y=data.language.values)):
        data.loc[v_, "kfold"] = fold_num
    return data

In [22]:
#full_train.reset_index(drop=True)
full_train.head()

,id,context,question,answer_text,answer_start,language,answers
0,5b2018761,मदर टेरेसा (२६ अगस्त १९१० - ५ सितम्बर १९९७) जि...,मदर टेरेसा की राष्ट्रीयता क्या थी?,भारतीय,328,hindi,"{'answer_start': [328], 'text': ['भारतीय']}"
1,7a3626dd9,இந்திய அரசியலமைப்பு (ஆங்கிலம்: Constitution of...,உலகின் மிகப்பெரிய அரசியலமைப்பு சட்டம் எது?,இந்தியா,62,tamil,"{'answer_start': [62], 'text': ['இந்தியா']}"
2,2d4a8e922,Main Page\n\nट्वाइलाइट एक रूमानी-फंतासी फ़िल...,2008 की ट्वाइलाइट रूमानी-फंतासी फ़िल्म में निक...,रोज़ाली हेल,4221,hindi,"{'answer_start': [4221], 'text': ['रोज़ाली हेल']}"
3,6ccfab41d,சாக்கிரட்டீசு (Socrates) (கி.மு 470/469 – கி....,சாக்ரடீஸ் எப்படி இறந்தார்?,விஷம்,8026,tamil,"{'answer_start': [8026], 'text': ['விஷம்']}"
4,422fd74bf,"इन्फोसिस लिमिटेड (BSE:, Nasdaq:) एक बहुराष्ट्र...",´इन्फोसिस लिमिटेड´ संस्थापक कौन है?,"इन्फोसिस की स्थापना २ जुलाई, १९८१ को पुणे में ...",453,hindi,"{'answer_start': [453], 'text': ['इन्फोसिस की ..."


In [23]:
full_train = create_folds(full_train, 2)
full_train.head()

,id,context,question,answer_text,answer_start,language,answers,kfold
0,5b2018761,मदर टेरेसा (२६ अगस्त १९१० - ५ सितम्बर १९९७) जि...,मदर टेरेसा की राष्ट्रीयता क्या थी?,भारतीय,328,hindi,"{'answer_start': [328], 'text': ['भारतीय']}",0
1,7a3626dd9,இந்திய அரசியலமைப்பு (ஆங்கிலம்: Constitution of...,உலகின் மிகப்பெரிய அரசியலமைப்பு சட்டம் எது?,இந்தியா,62,tamil,"{'answer_start': [62], 'text': ['இந்தியா']}",1
2,2d4a8e922,Main Page\n\nट्वाइलाइट एक रूमानी-फंतासी फ़िल...,2008 की ट्वाइलाइट रूमानी-फंतासी फ़िल्म में निक...,रोज़ाली हेल,4221,hindi,"{'answer_start': [4221], 'text': ['रोज़ाली हेल']}",1
3,6ccfab41d,சாக்கிரட்டீசு (Socrates) (கி.மு 470/469 – கி....,சாக்ரடீஸ் எப்படி இறந்தார்?,விஷம்,8026,tamil,"{'answer_start': [8026], 'text': ['விஷம்']}",0
4,422fd74bf,"इन्फोसिस लिमिटेड (BSE:, Nasdaq:) एक बहुराष्ट्र...",´इन्फोसिस लिमिटेड´ संस्थापक कौन है?,"इन्फोसिस की स्थापना २ जुलाई, १९८१ को पुणे में ...",453,hindi,"{'answer_start': [453], 'text': ['इन्फोसिस की ...",0


In [24]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


In [25]:
# Process the data
train = full_train[full_train.kfold != fold]
valid = full_train[full_train.kfold == fold]

train_data = Dataset.from_pandas(train)
valid_data = Dataset.from_pandas(valid)
test_data = Dataset.from_pandas(test)

# Prepare features
train_features = train_data.map(prepare_train_features, batched=True, remove_columns=train_data.column_names)
valid_features = valid_data.map(prepare_train_features, batched=True, remove_columns=valid_data.column_names)

# Negative sampling
train_str1 = train_features.map(negative_sampling, fn_kwargs={'ratio':0.1}, batched=True, batch_size=8)
train_str2 = train_features.map(negative_sampling, fn_kwargs={'ratio':0.2}, batched=True, batch_size=8)

from datasets import concatenate_datasets
tokenized_train_all = concatenate_datasets([train_str1, train_str2])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/612 [00:00<?, ?ba/s]

  0%|          | 0/612 [00:00<?, ?ba/s]

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained('5foldxlmr/output/checkpoint-fold-2/pytorch_model.bin')

args = TrainingArguments(
    f"chaii-qa-{EXP_NAME}",
    evaluation_strategy = "steps",
    logging_strategy = "steps",
    logging_steps = 100,
    save_steps = 600,
    save_strategy = "steps",
    learning_rate=3e-4,
    gradient_accumulation_steps=4,
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to='none',
    save_total_limit=15
)

data_collator = default_data_collator

trainer = MyTrainer(
    model,
    args,
    train_dataset=tokenized_train_all,
    eval_dataset=valid_features,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()
trainer.save_model(FOLDER)

***** Running training *****
  Num examples = 1979
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 186


Step,Training Loss,Validation Loss
100,5.449400,4.118545


***** Running Evaluation *****
  Num examples = 3918
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to chaii_fold_0
Configuration saved in chaii_fold_0/config.json
Model weights saved in chaii_fold_0/pytorch_model.bin
tokenizer config file saved in chaii_fold_0/tokenizer_config.json
Special tokens file saved in chaii_fold_0/special_tokens_map.json


In [ ]:
validation_features = valid_data.map(prepare_validation_features,batched=True, remove_columns=valid_data.column_names)

valid_feats_small = validation_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

max_answer_length = 30

examples = valid_data
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

references = [{"id": ex["id"], "context": ex["context"], "question": ex["question"], "answer": ex["answer_text"]} for ex in valid_data]

# Round 2 results
# final
raw_predictions = trainer.predict(valid_feats_small)
final_predictions = postprocess_qa_predictions(valid_data, validation_features, raw_predictions.predictions)
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res['postuned'] = res['prediction'].apply(postuning)
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
res['em'] = res[['answer', 'prediction']].apply(compute_exact_match, axis=1)
res['f1'] = res[['answer', 'prediction']].apply(compute_f1, axis=1)

print("Model results:\n")
print(f"Jaccard Score: {res.jaccard.mean()}\nJaccard Score after post-tuning:{res.pjaccard.mean()}\nEM Score: {res.em.mean()}\nF1 Score: {res.f1.mean()}")

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3569 [00:00<?, ?ex/s]

***** Running Prediction *****
  Num examples = 3569
  Batch size = 8


Post-processing 390 example predictions split into 3569 features.


  0%|          | 0/390 [00:00<?, ?it/s]

Model results:

Jaccard Score: 0.3032880775584396
Jaccard Score after post-tuning:0.30405730832767036
EM Score: 0.24615384615384617
F1 Score: 0.3322513758383324


In [ ]:
#!pip install kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 4.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=dc643f5dd8866917dd405404781a221a73277978fba62117ed1fbf7b8f33fbd0
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
! mkdir ~/.kaggle